# Python REST Live Coding

Goal of this live coding session is to use Python to communicate with an existing REST API to retrieve and send data.

## REST API

In this workshow we will use the `ToDo` and `User` endpoint of [JSON Placeholder](https://jsonplaceholder.typicode.com/) which is a free fake API for testing and prototyping REST clients.

### ToDo endpoint

The `ToDo` endpoint [`https://jsonplaceholder.typicode.com/todos`](https://jsonplaceholder.typicode.com/todos) returns 200 ToDos:

```json
[
  {
    "userId": 1,
    "id": 1,
    "title": "delectus aut autem",
    "completed": false
  },
...
]
```


### User endpoint

The `User` endpoint [`https://jsonplaceholder.typicode.com/users`](https://jsonplaceholder.typicode.com/users) returns 10 users:

```json
[
  {
    "id": 1,
    "name": "Leanne Graham",
    "username": "Bret",
    "email": "Sincere@april.biz",
    "address": {
      "street": "Kulas Light",
      "suite": "Apt. 556",
      "city": "Gwenborough",
      "zipcode": "92998-3874",
      "geo": {
        "lat": "-37.3159",
        "lng": "81.1496"
      }
    },
    "phone": "1-770-736-8031 x56442",
    "website": "hildegard.org",
    "company": {
      "name": "Romaguera-Crona",
      "catchPhrase": "Multi-layered client-server neural-net",
      "bs": "harness real-time e-markets"
    }
  },
...
]
```

## Installing REST Library

The Python library [Requests](https://docs.python-requests.org/en/latest/index.html) is a simple HTTP library that makes calling REST endpoints easy and simple.

Execute `pip install requests` to install the libary onto your Raspberry (or any other machine with Python).

In [2]:
# % is needed to install requests in Jupyter
%pip install requests


Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple
Note: you may need to restart the kernel to use updated packages.


## GET Requests

### GET flat objects

Call `todos/1` to get the ToDo with the ID `1`.

In [3]:
import requests
api_url = "https://jsonplaceholder.typicode.com/todos/1"
r_todo = requests.get(api_url)
r_todo_json = r_todo.json()  # return dict
r_todo_json


{'userId': 1, 'id': 1, 'title': 'delectus aut autem', 'completed': False}

`response.json()` turns the returned JSON response into a python dictionary.  
To make working with the ToDos easier, we want them to be objects of a `Todo` class.

The `Todo` class is a simple python class with only a `__init__` function (constructor) and the properties given by the response object:

In [4]:
class Todo(object):

    def __init__(self, userId: int, id: int, title: str, completed: bool):
        self.userId = userId
        self.id = id
        self.title = title
        self.completed = completed


To create a object of `Todo` we can call the initializer with the respective dict values:

In [5]:
todo = Todo(userId=r_todo_json["userId"],
            id=r_todo_json["id"],
            title=r_todo_json["title"],
            completed=r_todo_json["completed"])
todo.title


'delectus aut autem'

Or, much simpler, use the `**`-*unpacking operator* which "unpacks" the dictionary and provides them as keywords:

In [6]:
todo = Todo(**r_todo_json)
todo.title

'delectus aut autem'

This is what happens behind the scenes when unpacking a dictionary:

In [7]:
Todo(**r_todo_json)

gets unpacked to

In [8]:
Todo(userId=1, id=1, title='delectus aut autem', completed=False)

```python

```

### GET complex objects

In contrast to the `ToDo` response which has a flat JSON structure, the `User` response has multiple nested JSON objects.

In [9]:
import requests
api_url = "https://jsonplaceholder.typicode.com/users/1"
r_user = requests.get(api_url)
r_user_json = r_user.json()  # return dict
r_user_json

{'id': 1,
 'name': 'Leanne Graham',
 'username': 'Bret',
 'email': 'Sincere@april.biz',
 'address': {'street': 'Kulas Light',
  'suite': 'Apt. 556',
  'city': 'Gwenborough',
  'zipcode': '92998-3874',
  'geo': {'lat': '-37.3159', 'lng': '81.1496'}},
 'phone': '1-770-736-8031 x56442',
 'website': 'hildegard.org',
 'company': {'name': 'Romaguera-Crona',
  'catchPhrase': 'Multi-layered client-server neural-net',
  'bs': 'harness real-time e-markets'}}

Let's create your `User` python class with all nested entities.

In [10]:
class Geo(object):
    def __init__(self, lat: float, lng: float) -> None:
        # shorter way to init attributes
        self.lat, self.lng = float(lat), float(lng)


class Address(object):
    def __init__(self, street: str, suite: str, city: str, zipcode: str, geo: Geo) -> None:
        self.street = street
        self.suite = suite
        self.city = city
        self.zipcode = zipcode
        self.geo = geo


class Company(object):
    def __init__(self, name: str, catchPhrase: str, bs: str) -> None:
        self.name, self.catchPhrase, self.bs = name, catchPhrase, bs


class User(object):

    def __init__(self, id: int, name: str, username: str, email: str, address: Address, phone: str, website: str,
                 company: Company):
        self.id = id
        self.name = name
        self.username = username
        self.email = email
        self.address = address
        self.phone = phone
        self.website = website
        self.company = company

In [11]:
u = User(**r_user_json)

# AttributeError: 'dict' object has no attribute 'name'
# u.company.name

Pythons `json` package (which is used by the `requests` library) only allows for decoding the following python primitives ([see official documentation](https://docs.python.org/3/library/json.html#json-to-py-table)):

| JSON          | Python    |
|---------------|-----------|
| object        | dict      |
| array         | list      |
| string        | str       |
| number (int)  | int       |
| number (real) | float     |
| true          | True      |
| false         | False     |
| null          | None      |

So every JSON object gets decoded to a `dict` and our unpacking trick only works for the root level dictionary.  
Every other nested objects gets decoded to a dictionary and not to an object as we would like:

In [12]:
u.company

{'name': 'Romaguera-Crona',
 'catchPhrase': 'Multi-layered client-server neural-net',
 'bs': 'harness real-time e-markets'}

---

**NOTE:**

There are many ways to decode complex JSON/dicts into python objects, like

* relying on *duck typing* and use helper classes like `SimpleNamespace` or `NamedTuple` in combination with `object_hook` ([examples](https://pynative.com/python-convert-json-data-into-custom-python-object/))
* some custom generic base class implementation with type checking ([example](https://www.seanjohnsen.com/2016/11/23/pydeserialization.html))
* use code generators like [swagger-codegen](https://github.com/swagger-api/swagger-codegen) or [openapi-python-client](https://pypi.org/project/openapi-python-client/) to auto-generate the necessary classes (e.g. requests, responses, client) from an API specification (e.g. [OpenAPI/Swagger](https://swagger.io/specification/)).

In this workshop we keep it **plain python** and use **class methods** with some boilerplate code to decode complex dictionaries into our typed python classes.

---

To be able to decode the JSON dict into a class we use a class `from_json` method on each model class:

```python
class Address(object):
    # same __init__

    @classmethod
    def from_json(cls, json: dict):
        geo = Geo.from_json(json.pop('geo'))
        return cls(geo=geo, **json)
```

Complete classes updated with `from_json` methods:

In [13]:
class Geo(object):
    def __init__(self, lat: float, lng: float) -> None:
        self.lat, self.lng = float(lat), float(lng)

    @classmethod
    def from_json(cls, json):
        return cls(**json)


class Address(object):

    def __init__(self, street: str, suite: str, city: str, zipcode: str, geo: Geo) -> None:
        self.street = street
        self.suite = suite
        self.city = city
        self.zipcode = zipcode
        self.geo = geo

    @classmethod
    def from_json(cls, json: dict):
        geo = Geo.from_json(json.pop('geo'))
        return cls(geo=geo, **json)


class Company(object):
    def __init__(self, name: str, catchPhrase: str, bs: str) -> None:
        self.name, self.catchPhrase, self.bs = name, catchPhrase, bs

    @classmethod
    def from_json(cls, json: dict):
        return cls(**json)


class User(object):

    def __init__(self, id: int, name: str, username: str, email: str, address: Address, phone: str, website: str,
                 company: Company):
        self.id = id
        self.name = name
        self.username = username
        self.email = email
        self.address = address
        self.phone = phone
        self.website = website
        self.company = company

    @classmethod
    def from_json(cls, json: dict):
        address = Address.from_json(json.pop('address'))
        company = Company.from_json(json.pop('company'))
        return cls(address=address, company=company, **json)

In [14]:
u = User.from_json(r_user_json)
u.address.geo.lat

-37.3159

## POST Requests

To create objects using HTTP POST simply call `post(url, json)`. This passes the json dictionary as a JSON encoded HTTP body.

In [15]:
import requests
api_url = "https://jsonplaceholder.typicode.com/todos"

todo = Todo(userId=14, id=8, title='Do something', completed=False)

r = requests.post(api_url, json=vars(todo))
print(r.status_code)
r.json()

201


{'userId': 14, 'id': 201, 'title': 'Do something', 'completed': False}

## Authentication

`requests` has out-of-the-box support for various forms of [authentication](https://docs.python-requests.org/en/latest/user/authentication/).

For today's live coding and the Plant Health project in general, it is enough to talk about *[Basic Authentication](https://docs.python-requests.org/en/latest/user/authentication/)*.

For Basic authentication the username and password are sent `base64` encoded to the server in the HTTP header field `Authorization`:

`Authorization: Basic <credentials>`

where `<credentials>` is the `base64` encoded string resulting from concatenating the `username` and password wit a colon.

### Example

User `elvis` with password `passwd` has the following plain text `<credentials>`: `elvis:passwd` which gets `base64` encoded to `ZWx2aXM6cGFzc3dk`. The resulting `Authorization` header looks like this:

`Authorization: Basic ZWx2aXM6cGFzc3dk`

Fortunately we don't need to combine and encode the credentials ourselfs as the `requests` library sets the correct header automatically so we only need to specify the username and password.

Simply add the them with `HTTPBasicAuth('user', 'pwd')` or as tuple `('user', 'pwd')` to the requests:

In [19]:
from requests.auth import HTTPBasicAuth

auth_header = HTTPBasicAuth('elvis', 'passwd')

resp = requests.get('https://httpbin.org/basic-auth/elvis/passwd',
                    auth=auth_header)  # or auth=('elvis', 'passwd')
resp.json()


{'authenticated': True, 'user': 'elvis'}

# Live Coding

## Task 1

Try these samples on your Raspberry Pi to get familiar with the `requests` library.

Time: ~5 mins



## Task 2

Start your Java backend and try to connect to the REST API and GET and POST some resources.

Time: ~15 mins